In [4]:
import tensorflow as tf
import tensorflow_recommenders as tfrs
import tensorflow_datasets as tfds

In [5]:
ratings = tfds.load('movielens/100k-ratings', split='train')
movies = tfds.load('movielens/100k-movies', split='train')

movies = movies.map(lambda x: {**x, 'movie_title_text': x['movie_title']})
ratings = ratings.map(lambda x: {**x, 'movie_title_text': x['movie_title']})

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-movies/0.1.1.incomplete8QF1LO/movielens-train.tfrecord*...:…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.


In [6]:
import numpy as np

all_ratings = list(ratings.map(lambda x: {
    'movie_title': x["movie_title"],
    'user_id': x['user_id'],
    'bucketized_user_age': x['bucketized_user_age'],
    'movie_genres': x['movie_genres'],
    'timestamp': x['timestamp'],
}).apply(tf.data.experimental.dense_to_ragged_batch(len(ratings))))[0]

all_movies = list(movies.apply(tf.data.experimental.dense_to_ragged_batch(len(movies))))[0]

unique_movie_titles = np.unique(all_movies['movie_title'])
unique_movie_genres, _ = tf.unique(all_movies['movie_genres'].flat_values)

max_timestamp = all_ratings['timestamp'].numpy().max()
min_timestamp = all_ratings['timestamp'].numpy().min()

unique_user_ids = np.unique(all_ratings['user_id'])

additional_features = {
    'timestamp_buckets': np.linspace(min_timestamp, max_timestamp, num=1000),
    'unique_movie_genres': tf.cast(unique_movie_genres, tf.int32),
    'bucketized_user_age': all_ratings['bucketized_user_age']
}
additional_features

Instructions for updating:
Use `tf.data.Dataset.ragged_batch` instead.


{'timestamp_buckets': array([8.74724710e+08, 8.74743291e+08, 8.74761871e+08, 8.74780452e+08,
        8.74799032e+08, 8.74817613e+08, 8.74836193e+08, 8.74854774e+08,
        8.74873354e+08, 8.74891935e+08, 8.74910515e+08, 8.74929096e+08,
        8.74947676e+08, 8.74966257e+08, 8.74984837e+08, 8.75003418e+08,
        8.75021998e+08, 8.75040579e+08, 8.75059159e+08, 8.75077740e+08,
        8.75096320e+08, 8.75114901e+08, 8.75133481e+08, 8.75152062e+08,
        8.75170642e+08, 8.75189223e+08, 8.75207803e+08, 8.75226384e+08,
        8.75244964e+08, 8.75263545e+08, 8.75282125e+08, 8.75300706e+08,
        8.75319286e+08, 8.75337867e+08, 8.75356447e+08, 8.75375028e+08,
        8.75393608e+08, 8.75412189e+08, 8.75430769e+08, 8.75449350e+08,
        8.75467930e+08, 8.75486511e+08, 8.75505091e+08, 8.75523672e+08,
        8.75542252e+08, 8.75560833e+08, 8.75579413e+08, 8.75597994e+08,
        8.75616574e+08, 8.75635155e+08, 8.75653735e+08, 8.75672316e+08,
        8.75690896e+08, 8.75709477e+08, 8.7

In [33]:
MOVIE_FEATURES = ['movie_title', 'movie_genres', 'movie_title_text']
USER_FEATURES = ['timestamp', 'bucketized_user_age']

from ttnn import *

In [34]:
trainset = (ratings.take(80_000)
                   .shuffle(100_000)
                   .apply(tf.data.experimental.dense_to_ragged_batch(2048))
                   .cache())

testset = (ratings.skip(80_000)
                  .take(20_000)
                  .apply(tf.data.experimental.dense_to_ragged_batch(2048))
                  .cache())
layer_sizes = [32]
embedding_size = 32
model = TwoTowerModel(layer_sizes, movies, unique_movie_titles, unique_user_ids,
                               embedding_size,
                               additional_features=additional_features)

model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1), run_eagerly=True)
model_history = model.fit(
    trainset,
    validation_data=testset,
    validation_freq=1,
    epochs=20,
    verbose=1)

Epoch 1/20
40/40 [==============================] - 34s 801ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 14474.2144 - regularization_loss: 0.0000e+00 - total_loss: 14474.2144 - val_factorized_top_k/top_1_categorical_accuracy: 0.0011 - val_factorized_top_k/top_5_categorical_accuracy: 0.0089 - val_factorized_top_k/top_10_categorical_accuracy: 0.0202 - val_factorized_top_k/top_50_categorical_accuracy: 0.1164 - val_factorized_top_k/top_100_categorical_accuracy: 0.2270 - val_loss: 10953.9824 - val_regularization_loss: 0.0000e+00 - val_total_loss: 10953.9824
Epoch 2/20
40/40 [==============================] - 19s 468ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.000

In [35]:
user_id = '42'
rating_idx = tf.where(all_ratings['user_id'] ==
                                  user_id.encode()).numpy().squeeze()[0]

user_details_for_query = {'user_id': np.array(['42']),
                          **{col_name: np.array([all_ratings[col_name][rating_idx].numpy()])
                             for col_name in USER_FEATURES}}
user_details_for_query

index = tfrs.layers.factorized_top_k.BruteForce()
index.index_from_dataset(movies
                         .apply(tf.data.experimental.dense_to_ragged_batch(100)).map(model.candidate_model))
_, titles = index(model.query_model(user_details_for_query))
title_names = np.array([unique_movie_titles[x] for x in titles.numpy().squeeze()])
print(f"Recommendations for user {user_id}:\n"
                  f" {title_names[:10]}")

Recommendations for user 42:
 [b'Lamerica (1994)' b'Hard Target (1993)' b'Trees Lounge (1996)'
 b'FairyTale: A True Story (1997)' b'Squeeze (1996)' b'Candyman (1992)'
 b'Sliding Doors (1998)' b'Program, The (1993)' b'New York Cop (1996)'
 b'Tomorrow Never Dies (1997)']
